<h1> 1 exercicio <h1/>

In [ ]:
import cmath
import numpy
import math
from cmath import phase 
import sympy as sym
from numpy import exp, abs, angle, conjugate
import numpy as np
from scipy.constants import mu_0, epsilon_0
from scipy.special import k1,k0,i1,i0
from mpmath import *
mp.dps = 25; mp.pretty = True

#faz o print das matrizes ficar bonitinho
np.set_printoptions(linewidth=500)

#potencia base do sistema
Sb=100*10**6

#impedancia base
Zb = ((138*10**3)**2)/(100*10**6)

#dados gerador
Sg = 250*10**6
Xg = 8/100

#dados transformador
St = 250*10**6
Z = 0.1
Xt=10/100

#dados circuito
l = 120*10**3
V = 138*10**3

#dados carga
Pc = 40*10**6
#cenario 1 (noite) e cenario 2 (dia)
Fp = ((-1)*0.92, 0.92)
#tensao na carga
Vc=(0.95,1.05)

#configuração da rede
Rdcf = 0.17*10**(-3)
Rdcpr = 0.17*10**(-3)
rf = 18.29/(2*10**(-3))
rpr = 4.57*10**(-3)
xc = (-3.8, 3.8, -3.8, 0)
yc = (15.033, 16.933, 18.833, 27.467)
fl = (9.25, 9.25, 9.25 , 4.7)
L=120 #km

#altura media
ycm = []
for i in range(len(yc)):
    ycm.append(yc[i] - (2/3)*fl[i])

ncond = len(xc)
nfases = 3
npr =1 
nc = ncond

#Circuito 138 kV (penguin)
rf = 7.1501*10**(-3)
rint = 2.3876*10**(-3)
Rdc = 35.356020838304296*10**(-5)
rhoc = Rdc* math.pi*(rf**2 - rint**2)

#circuito 3/8 EHS (C.G.)
rpr = 4.57*10**(-3)
Rdcpr = 4.19*10**(-3)
rhoc_pr = Rdcpr*math.pi*(rpr**2)


#dados adicionais
epsilon_r = 10
sigma_s = 1*10**(-3)

#frquencia
f = 60

In [ ]:
#----funcoes para os calculos-----
omega = 2*math.pi*f
ri = rint + 10**(-6)
gama_S = np.sqrt(1j*omega*mu_0*(sigma_s + 1j*omega*epsilon_r*epsilon_0))
gama_ar = 1j*omega*np.sqrt(mu_0*epsilon_0)
eta = np.sqrt(gama_S**2 - gama_ar**2)


def S1(xc,npr,yc,rf,rpr):
    ncond=len(xc)
    s1=np.eye(ncond)*1j
    for i in range(ncond):
        for j in range(ncond):
            if(i != j):
                s1[i,j] = log(1 + (2/(eta*np.sqrt((xc[i] - xc[j])**2 + (yc[i]+yc[j])**2))))
            elif (i+1) <= nc-npr:
                s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rf**2))))
            else:
                s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rpr**2))))
    return s1

def Mpot(xc,yc,npr,rf,rpr):
    ncond=len(xc)
    pot=np.eye(ncond)*1j
    for i in range(ncond):
        for j in range(ncond): 
            if i!=j: #entre fases
                num=(xc[i]-xc[j])**2+(yc[i]+yc[j])**2
                den=(xc[i]-xc[j])**2+(yc[i]-yc[j])**2
                pot[i,j] = 0.5*log(num/den);
            elif (i+1)<= (ncond-npr): #condutor de fase
                pot[i,j] = math.log((2*(yc[i])/rf))
            else: #para-raio
                pot[i,j] = log((2*(yc[i])/rpr))
    return pot

def zexternal(omega,rho,npr,xc,yc,rf,rpr):
        p = math.sqrt(rho/(1j*omega*mu))
        ncond= len(xc)
        zout=np.eye(ncond)*1j
        for i in range(ncond):
            for j in range(ncond): 
                if i!=j: #entre fases
                    num = (xc[i]-xc[j]**2)+(2*p+yc[i]+yc[j]**2)
                    den = (xc[i]-xc[j]**2)+(yc[i]-yc[j]**2)
                    zout[i,j] = 1j*omega*mu_0/(4*pi)*log(num/den)
                elif (i+1)<= (ncond-npr):
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rf)
                else:
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rpr)
        return zout
       
def Zint(omega,rhoc,rf):
    etac = np.sqrt(1j*omega*mu_0/rhoc)

    zint = rhoc*etac/(2*math.pi*rf)*besseli(0,etac*rf)/besseli(1,etac*rf)
    return zint

def Zinttub(omega,rhoc,rf,ri ):
    etac = np.sqrt(1j*omega*mu_0/rhoc);

    num =  besseli(0,etac*rf)*besselk(1,etac*ri) + besselk(0,etac*rf)*besseli(1,etac*ri) 


    den =  besseli(1, etac*rf)*besselk(1,etac*ri) - besseli(1,etac*ri)*besselk(1,etac*rf)

    zin = rhoc*etac/(2*math.pi*rf)*num/den
    return zin

def Zin(xc,npr,rhoc,rhoc_pr,rf,ri):
    ncond= len(xc);
    zin=np.eye(ncond)*1j

    for i in range(ncond):
        for j in range(ncond): 
            if i!=j: #entre fases
                zin[i,j]=0
            elif (i+1)<= (ncond-npr):
                zin[i,j]=Zinttub(omega,rhoc,rf,ri)
            else:
                zin[i,j]=Zint(omega,rhoc_pr,rpr)
    return zin

In [ ]:
#-------calculo dos parametros-----

z1 = Zin(xc,npr,rhoc,rhoc_pr,rf,ri) + (((1j*omega*mu_0)/2/math.pi) * (Mpot(xc,yc,npr,rf,rpr) + S1(xc,npr,yc,rf,rpr)))

y1 =  1j*omega*2*math.pi*epsilon_0*(np.linalg.inv(Mpot(xc,yc,npr,rf,rpr))) + 3.0*10**(-11)*np.eye(ncond)



<h1> Matriz de impedância (longitudinal) por unidade de comprimento em Ohm/km<h1/>



In [ ]:
z1=z1*1000
print(z1)

[[ 412464.35718767 +989740.3262756j    58620.68639375 +446657.89695471j   58602.47197073 +501223.96713588j   58443.92315262 +408639.65420869j]
 [  58620.68639375 +446657.89695471j  412395.71034691 +989810.03945319j   58553.79969187 +446725.91156382j   58409.92360109 +419931.75895404j]
 [  58602.47197073 +501223.96713588j   58553.79969187 +446725.91156382j  412327.19032754 +989879.75043199j   58375.94476192 +432899.96829897j]
 [  58443.92315262 +408639.65420869j   58409.92360109 +419931.75895404j   58375.94476192 +432899.96829897j 4248252.43959586+1027380.50775642j]]


<h1> Matriz de admitância (longitudinal) por unidade de comprimento em Ohm/km<h1/>



In [ ]:
y1=y1*1000
print(y1)

[[3.e-05+0.00275694j 0.e+00-0.00032396j 0.e+00-0.00060869j 0.e+00-0.00019784j]
 [0.e+00-0.00032396j 3.e-05+0.00264198j 0.e+00-0.00033833j 0.e+00-0.00029001j]
 [0.e+00-0.00060869j 0.e+00-0.00033833j 3.e-05+0.00272612j 0.e+00-0.00033589j]
 [0.e+00-0.00019784j 0.e+00-0.00029001j 0.e+00-0.00033589j 3.e-05+0.00235717j]]


<h1> Matriz de sequencia<h1/>



In [ ]:
#Retiramos as info do pararaio das matrizes
Zabc=z1[0:3,0:3]
Yabc=y1[0:3,0:3]

#matrizes de sequencia
a = complex(-0.5,0.866025)
a2 = a**2
A = np.array([[1, 1, 1], [1, a2, a], [1, a, a2]])
z012 = np.linalg.inv(A)@Zabc@A
y012 = np.linalg.inv(A)@Yabc@A

In [ ]:
print(z012)

[[ 0.52958039+1.91954855j -0.01571837+0.0090121j   0.01582558+0.00906219j]
 [ 0.01582556+0.00906185j  0.35380344+0.52494077j  0.0314794 -0.01818747j]
 [-0.01571844+0.00901179j -0.03148783-0.01816894j  0.35380342+0.52494079j]]


In [ ]:
print(y012)

[[ 3.00000000e-08+1.86102944e-06j  5.79068286e-08-1.83661813e-08j -5.79068286e-08-1.83662281e-08j]
 [-5.79069266e-08-1.83658680e-08j  2.99999425e-08+3.13200450e-06j -1.88676395e-07+1.09622299e-07j]
 [ 5.79069266e-08-1.83659148e-08j  1.88676395e-07+1.09622299e-07j  3.00000575e-08+3.13200435e-06j]]


In [ ]:
#os parametros unitarios de sequencia+ entao sao:
z=z012[1][1]
y=y012[1][1]
#em pu:
z_pu=(z/Zb)*L
y_pu=(y*Zb)*L #Yb=1/Zb

(0.0006855826863124067+0.07157507249136225j)

In [ ]:
#Funcao que recebe a tensao em pu, a potencia ativa e o fator de potencia e nos
#retorna o valor abs da tensao e da corrente na fonte
def calcular(V_pu,P,fp_pu): 
    S=P*fp_pu
    S_pu= abs(S/Sb)
    if fp_pu>0: # Indutivo
      angulo=-acos(abs(fp_pu))
    else: #Capacitivo
      angulo=acos(abs(fp_pu))

    I_pu=complex(S_pu/V_pu*cos(angulo),S_pu/V_pu*sin(angulo))
    print(I_pu)
    
    Carg = np.array([[V_pu],[I_pu]])
    
    Impedancia_Gera = np.array([[1,Xg],
                                [0,1]])
    
    Impedancia_Transfo = np.array([[1,Xt],
                                  [0,1]])
    
    Impedancia_Linha =   np.array([[1+(z_pu*y_pu)/2     ,z_pu],
                               [y_pu*(1+(z_pu*y_pu)/4),1+(z_pu*y_pu)/2]])
    
    Impedancia_Total = Impedancia_Gera @ Impedancia_Transfo @ Impedancia_Linha
    
    print(Impedancia_Linha)
    Font = Impedancia_Total@Carg
    print("V em pu valor absoluto na fonte:")
    print(abs(Font[0]))
    print("I em pu valor absoluto na fonte:")
    print(abs(Font[1]))

In [ ]:
sin(acos(-0.92))

0.3919183588453084018895085

In [ ]:
for i in Vc:
    for j in Fp:
        if j >0:
          tex=" Indutivo"
        else:
          tex=" Capacitivo"
        print("Utilizando o valor de V na carga = "+str(i)+" e fator de potencia = "+str(abs(j))+tex)
        calcular(i,Pc,j)
        print("")

Utilizando o valor de V na carga = 0.95 e fator de potencia = 0.92 Capacitivo
(0.3563789473684211+0.15181679584744578j)
[[9.99988239e-01+8.09180765e-06j 2.22938528e-04+3.30775532e-04j]
 [6.85289069e-04+7.15746544e-02j 9.99988239e-01+8.09180765e-06j]]
V em pu valor absoluto na fonte:
[1.01506014]
I em pu valor absoluto na fonte:
[0.41926679]

Utilizando o valor de V na carga = 0.95 e fator de potencia = 0.92 Indutivo
(0.3563789473684211-0.15181679584744578j)
[[9.99988239e-01+8.09180765e-06j 2.22938528e-04+3.30775532e-04j]
 [6.85289069e-04+7.15746544e-02j 9.99988239e-01+8.09180765e-06j]]
V em pu valor absoluto na fonte:
[1.01449418]
I em pu valor absoluto na fonte:
[0.36673347]

Utilizando o valor de V na carga = 1.05 e fator de potencia = 0.92 Capacitivo
(0.32243809523809525+0.13735805338578427j)
[[9.99988239e-01+8.09180765e-06j 2.22938528e-04+3.30775532e-04j]
 [6.85289069e-04+7.15746544e-02j 9.99988239e-01+8.09180765e-06j]]
V em pu valor absoluto na fonte:
[1.10884663]
I em pu valor ab

<h1> 2 exercicio <h1/>

In [ ]:
import numpy as np
S=Zabc@Yabc
autovalor, autovetor = np.linalg.eig(S)
h1=autovalor[0]
h2=autovalor[1]
h3=autovalor[2]

def avalues(matrix,h):
    a11=1
    #ok
    a12=matrix[0][1]/(matrix[0][0]-h)
    #ok
    a13=matrix[0][2]/(matrix[0][0]-h)
    #ok
    a21=0
    #ok
    a22 = (((-1*matrix[0][1]*matrix[1][0])/(matrix[0][0] - h)) +(matrix[1][1] - h))*(((-1*matrix[0][1]*matrix[2][0])/(matrix[0][0] - h))+matrix[2][1])
    #ok
    a23 = (((-1*matrix[0][2]*matrix[1][0])/(matrix[0][0] - h)) +matrix[1][2])*(((-1*matrix[0][1]*matrix[2][0])/(matrix[0][0] - h))+(matrix[1][1] - h))
    a31=0
    a32=0
    #ok
    a33 = (((-1*matrix[0][2]*matrix[1][0])/(matrix[0][0] - h)) + matrix[1][2] )*(((-1*matrix[0][1]*matrix[2][0])/(matrix[0][0] - h))+matrix[2][1]) + (((-1*matrix[0][2]*matrix[2][0])/(matrix[0][0] - h)) +(matrix[2][2] - h))*(((-1*matrix[0][1]*matrix[1][0])/(matrix[0][0] - h))+ (matrix[1][1] - h))

    p = 1/np.sqrt(1 + (a23**2)/(a22**2) + (((a12**2)*(a23**2)-2*a12*a13*a22*a23+(a12**2)*(a13**2))/((a11**2)*(a22**2))))

    Tv1 = ((-1)*((a12*a23)/a22)*p)/a11 + (a13*p)/a11
    Tv2 = (a23/a22)*p
    Tv3 = -p
    return (Tv1,Tv2,Tv3)
    
TV=np.array([[avalues(S,h1)[0],avalues(S,h2)[0],avalues(S,h3)[0]],[avalues(S,h1)[1],avalues(S,h2)[1],avalues(S,h3)[1]],[avalues(S,h1)[2],avalues(S,h2)[2],avalues(S,h3)[2]]])
T_gama=np.linalg.inv(TV)
T_gamaT=np.transpose(T_gama)

In [ ]:
Zm=T_gama@Zabc@T_gamaT
Ym=T_gama@Yabc@TV
Zm


array([[ 1.01446943e+33-3.13681614e+33j,  4.62688564e+36+3.03140656e+37j,
        -2.15486216e+35-2.83204939e+35j],
       [ 4.62688564e+36+3.03140656e+37j, -1.58443343e+41-2.32262208e+41j,
         2.84818348e+39+1.43514512e+39j],
       [-2.15486216e+35-2.83204939e+35j,  2.84818348e+39+1.43514512e+39j,
        -3.57982020e+37+2.83481527e+36j]])

In [ ]:
from sympy import besseli, besselk, Matrix
import sympy as sym
import numpy as np
#transposicao de fases
Z6, Y6, Z3, Y3 = sym.symbols('Z6, Y6, Z3, Y3')

R = np.array([[0., 1., 0.],[0., 0., 1.],[1., 0., 0.]])
O = np.zeros((3, 3))
F1 = np.concatenate((R,O), axis=1)
F2 = np.concatenate((O,R), axis=1)
F = np.concatenate((F1,F2), axis=0)


Q1 = Matrix([[1+ (Z6*Y6)/2 , Z6],[Y6*(1 + (Z6*Y6)/4),(1+ (Z6*Y6)/2)]])
Q2 = Matrix([[1+ (Z3*Y3)/2 , Z3],[Y3*(1 + (Z3*Y3)/4),(1+ (Z3*Y3)/2)]])

#LT = Q1*F*Q2*F*Q2*F*Q1

F

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.]])

In [ ]:
Ym11 = ((Tv11)**2)*y11 + ((Tv21)**2)*y22 + ((Tv31)**2)*y33 + 2*(Tv11*Tv21*y12) + 2*(Tv11*Tv31*y13) + 2*(Tv21*Tv31*y23)
Zm11 = ((Tv11)**2)*y11 + ((Tv21)**2)*y22 + ((Tv31)**2)*y33 + 2*(Tv11*Tv21*y12) + 2*(Tv11*Tv31*y13) + 2*(Tv21*Tv31*y23)
gama_m1 = np.sqrt(Zm1*Ym1)

F1 = np.cosh(gama_m1*d) * eye(3)
F2 = (Zcm1*(np.senh(gama_m1*d))) * eye(3)
F3 = ((1/Zcm1)*(np.senh(gama_m1*d))) * eye(3)
F4 = -1*F1

<h1> 3 exercicio <h1/>

In [ ]:
import cmath
import numpy
import math
from cmath import phase 
import sympy as sym
from numpy import exp, abs, angle, conjugate
import numpy as np
from scipy.constants import mu_0, epsilon_0
from scipy.special import k1,k0,i1,i0
from sympy import besseli, besselk, Matrix

#caso 1 
P1 = 60*10**6

P2 = 80*10**6

#circuito 3/8 EHS (C.G.)
rpr = 4.57*10**(-3)
Rdcpr = 4.19*10**(-3)
resist_pr = Rdcpr*math.pi*(rpr**2)

#circuito (Dove)
ri_Dove = 6,988 * 10**(-3)
rf_Dove = 11.773* 10**(-3)
rhoc_Dove = 29.567*10**(-9)


Tcl = np.matrix([[-1/np.sqrt(6), 2/np.sqrt(6), -1/np.sqrt(6)],[1/np.sqrt(2), 0., -1./np.sqrt(2)],[1./np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)]])
Tcl**-1


Ym = []
Zm = []

YMab0 = Tcl*Ym*(Tcl**-1)
ZMab0 = Tcl*Ym*(Tcl**-1)

Ymd = np.matrix([[YMab0, 0],[0, YMab0]])
Zmd = np.matrix([[ZMab0, 0],[0, ZMab0]])

#-----Circuito Duplo------

xc = (-3.4, 3.4, -3.4, 3.4, 3.4, 3.4, 0) 
yc = (12.3333, 16.3333, 20.3333, 12.3333, 16.3333, 20.3333, 30.7667) 
npr=1
#Calculo dos parametros
z1 = Zin(xc,npr,rhoc_Dove,rhoc_pr,rf_Dove,ri_Dove) + (((1j*omega*mu_0)/2/math.pi) * (Mpot(xc,yc,npr,rf_Dove,rpr) + S1(xc,npr,yc,rf_Dove,rpr)))

elem1 = np.linalg.inv(A)@Zabc@A
elem2 = np.linalg.inv(A)@(Zabc - Zabc_)@A
elem3 = np.linalg.inv(A)@(Zabc_ - Zabc)@A
elem4 = np.linalg.inv(A)@Zabc_@A
np.matrix([[elem1, elem2],[elem3, elem4]])

#-----Dois circuitos identicos------

ValueError: ignored